In [1]:
import json
from bs4 import BeautifulSoup
import requests
import time
import os

In [2]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

In [3]:
def extract_luas_tanah(luas_tanah):
    luas = luas_tanah.split(' ')
    if(luas[0].__contains__(',')):
        return luas[0].replace(',','')
    return float(luas[0])

def extract_harga_juta(harga):
    hargasplit = harga.split(' ')
    return float(hargasplit[1].replace(',', ''))

def extract_harga_miliar(harga):
    hargasplit = harga.split(' ')
    return float(hargasplit[1])

def extract_kota(lokasi):
    return lokasi.split(',')[0]

def extract_provinsi(lokasi):
    return (lokasi.split(',')[1])[1:]

In [4]:
def scraping_pages(object, firstPage, lastPage):
    count_pages = 1
    for page in range(firstPage, lastPage+1):
        n = 0
        pages = requests.get("https://www.dotproperty.id/houses-for-sale?page="+str(page), headers=headers)
        soup = BeautifulSoup(pages.text, 'html.parser')
        properti = soup.find_all("div", "search-list normal")
        for item in properti:
            tag = item.find('div', 'right-block')
            title = item.a.get('title')
            link = item.a.get('href')
            
            harga = tag.find('div', 'price').text.strip()
            unit = ''
            try:
                if (harga.__contains__('Milyar')):
                    unit = 'Miliar'
                    hargaint = extract_harga_miliar(harga)
                else:
                    unit = 'Juta'
                    hargaint = extract_harga_juta(harga)/10**6
            except ValueError:
                unit = ''
                hargaint = ''
            
            nama = tag.h3.text.strip()
            lokasi = tag.small.text
            if (lokasi.__contains__(',')):
                kota = extract_kota(lokasi)
                provinsi = extract_provinsi(lokasi)
            else:
                kota = ''
                provinsi = extract_kota(lokasi)
            tipe = tag.find('div', 'hidden-property-type hidden').text
            #Accomodation
            ul = tag.find('ul')
            acc = []
            for li in ul.find_all('li'):
                acc.append(li.span.text)
            if(len(acc) == 3):
                kamar_mandi = int(acc[0])
                kamar_tidur = int(acc[1])
                luas_tanah = acc[2]
                luas_int = extract_luas_tanah(luas_tanah)
            else:
                acckamar = []
                for item in acc:
                    if(item.__contains__('m2')):
                        luas_tanah = item
                        luas_int = extract_luas_tanah(luas_tanah)
                    else:
                        acckamar.append(item)
                        luas_tanah = ''
                if(len(acckamar) == 2):
                    kamar_tidur = int(acckamar[0])
                    kamar_mandi = int(acckamar[1])
                elif(len(acckamar) == 1):
                    kamar_tidur = int(acckamar[0])
                    kamar_mandi = ''
                elif(len(acckamar) == 0):
                    kamar_tidur = ''
                    kamar_mandi = ''

            value = {
                "Title": title, 
                "Nama": nama,
                "Tipe": tipe,
                "Jumlah Kamar Mandi": kamar_mandi,
                "Jumlah Kamar Tidur": kamar_tidur, 
                "Luas Tanah(m2)": luas_int,
                "Harga": hargaint,
                "Unit Harga": unit,
                "Kota": kota,
                "Provinsi": provinsi,
                "Link": link
                    }
            object.append(value)
            n += 1
        print("Berhasil melakukan scraping halaman ke-"+str(count_pages)+" sebanyak "+str(n)+" data.")
        count_pages += 1
        time.sleep(1)

In [5]:
values = []
scraping_pages(values, 1, 50)

Berhasil melakukan scraping halaman ke-1 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-2 sebanyak 20 data.
Berhasil melakukan scraping halaman ke-3 sebanyak 1 data.
Berhasil melakukan scraping halaman ke-4 sebanyak 2 data.
Berhasil melakukan scraping halaman ke-5 sebanyak 20 data.
Berhasil melakukan scraping halaman ke-6 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-7 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-8 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-9 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-10 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-11 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-12 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-13 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-14 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-15 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-16 sebanyak 25 data.
Berhasil melakukan scraping halaman ke-17 sebanyak 

In [6]:
filename = input("Nama File: ")
filepath = 'G:/My Drive/Tugas Lab Basis Data/Seleksi-2022-Tugas-1/Data Scraping/data'
dump = json.dumps(values, indent=4)
filepath = os.path.join(filepath, filename + ".json")
with open(filepath, "w") as file:
    file.write(dump)
    file.close()

Nama File:  DataPropertyScrape
